# <img style="float: left; padding-right: 10px; width: 45px" src="https://github.com/Harvard-IACS/2018-CS109A/blob/master/content/styles/iacs.png?raw=true"> CS109A Introduction to Data Science 

## Lab 2: Pandas and Web Scraping with Beautiful Soup

**Harvard University**<br>
**Fall 2019**<br>
**Instructors:** Pavlos Protopapas, Kevin Rader, and Chris Tanner<br>
**Lab Instructors:** Chris Tanner and Eleni Kaxiras <br>
**Authors:** Rahul Dave, David Sondak, Will Claybaugh, Pavlos Protopapas, Chris Tanner, Eleni Kaxiras

---

In [4]:
## RUN THIS CELL TO GET THE RIGHT FORMATTING 
from IPython.core.display import HTML
def css_styling():
    styles = open("../../styles/cs109.css", "r").read()
    return HTML(styles)
#css_styling()

# Table of Contents 
<ol start="0">
<li> Learning Goals </li>
<li> Loading and Cleaning with Pandas</li>
<li> Parsing and Completing the Dataframe  </li>
<li> Grouping </li>
</ol>

## Learning Goals

About 6,000 odd "best books" were fetched and parsed from [Goodreads](https://www.goodreads.com). The "bestness" of these books came from a proprietary formula used by Goodreads and published as a list on their web site.

We parsed the page for each book and saved data from all these pages in a tabular format as a CSV file. In this lab we'll clean and further parse the data.  We'll then do some exploratory data analysis to answer questions about these best books and popular genres.  


By the end of this lab, you should be able to:

- Load and systematically address missing values, ancoded as `NaN` values in our data set, for example, by removing observations associated with these values.
- Parse columns in the dataframe to create new dataframe columns.
- Use groupby to aggregate data on a particular feature column, such as author.

*This lab corresponds to lectures #1, #2, and #3 and maps on to homework #1 and further.*

### Basic EDA workflow

(From the lecture, repeated here for convenience).

The basic workflow is as follows:

1. **Build** a DataFrame from the data (ideally, put all data in this object)
2. **Clean** the DataFrame. It should have the following properties:
    - Each row describes a single object
    - Each column describes a property of that object
    - Columns are numeric whenever appropriate
    - Columns contain atomic properties that cannot be further decomposed
3. Explore **global properties**. Use histograms, scatter plots, and aggregation functions to summarize the data.
4. Explore **group properties**. Use groupby and small multiples to compare subsets of the data.

This process transforms your data into a format which is easier to work with, gives you a basic overview of the data's properties, and likely generates several questions for you to followup in subsequent analysis.

## Part 1: Loading and Cleaning with Pandas 
Read in the `goodreads.csv` file, examine the data, and do any necessary data cleaning. 

Here is a description of the columns (in order) present in this csv file:

```
rating: the average rating on a 1-5 scale achieved by the book
review_count: the number of Goodreads users who reviewed this book
isbn: the ISBN code for the book
booktype: an internal Goodreads identifier for the book
author_url: the Goodreads (relative) URL for the author of the book
year: the year the book was published
genre_urls: a string with '|' separated relative URLS of Goodreads genre pages
dir: a directory identifier internal to the scraping code
rating_count: the number of ratings for this book (this is different from the number of reviews)
name: the name of the book
```

Let us see what issues we find with the data and resolve them.  



----




After loading appropriate libraries


In [5]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)

### Cleaning: Reading in the data
We read in and clean the data from `goodreads.csv`.

In [6]:
#Read the data into a dataframe
df = pd.read_csv("../data/goodreads.csv", encoding='utf-8')

#Examine the first few rows of the dataframe
df.head()

,4.40,136455,0439023483,good_reads:book,https://www.goodreads.com/author/show/153394.Suzanne_Collins,2008,/genres/young-adult|/genres/science-fiction|/genres/dystopia|/genres/fantasy|/genres/science-fiction|/genres/romance|/genres/adventure|/genres/book-club|/genres/young-adult|/genres/teen|/genres/apocalyptic|/genres/post-apocalyptic|/genres/action,dir01/2767052-the-hunger-games.html,2958974,"The Hunger Games (The Hunger Games, #1)"
0,4.41,16648,0439358078,good_reads:book,https://www.goodreads.com/author/show/1077326....,2003.0,/genres/fantasy|/genres/young-adult|/genres/fi...,dir01/2.Harry_Potter_and_the_Order_of_the_Phoe...,1284478,Harry Potter and the Order of the Phoenix (Har...
1,3.56,85746,0316015849,good_reads:book,https://www.goodreads.com/author/show/941441.S...,2005.0,/genres/young-adult|/genres/fantasy|/genres/ro...,dir01/41865.Twilight.html,2579564,"Twilight (Twilight, #1)"
2,4.23,47906,0061120081,good_reads:book,https://www.goodreads.com/author/show/1825.Har...,1960.0,/genres/classics|/genres/fiction|/genres/histo...,dir01/2657.To_Kill_a_Mockingbird.html,2078123,To Kill a Mockingbird
3,4.23,34772,0679783261,good_reads:book,https://www.goodreads.com/author/show/1265.Jan...,1813.0,/genres/classics|/genres/fiction|/genres/roman...,dir01/1885.Pride_and_Prejudice.html,1388992,Pride and Prejudice
4,4.25,12363,0446675539,good_reads:book,https://www.goodreads.com/author/show/11081.Ma...,1936.0,/genres/classics|/genres/historical-fiction|/g...,dir01/18405.Gone_with_the_Wind.html,645470,Gone with the Wind


Oh dear. That does not quite seem to be right. We are missing the column names. We need to add these in! But what are they?

Here is a list of them in order:

`["rating", 'review_count', 'isbn', 'booktype','author_url', 'year', 'genre_urls', 'dir','rating_count', 'name']`

<div class="exercise"><b>Exercise</b></div>
Use these to load the dataframe properly! And then "head" the dataframe... (you will need to look at the read_csv docs)


In [8]:
# your code here
df=pd.read_csv("../data/goodreads.csv", header=None,
               names=["rating", 'review_count', 'isbn', 'booktype','author_url', 'year', 'genre_urls', 'dir','rating_count', 'name'],
)

#Examine the first few rows of the dataframe
df.head()

,rating,review_count,isbn,booktype,author_url,year,genre_urls,dir,rating_count,name
0,4.40,136455,0439023483,good_reads:book,https://www.goodreads.com/author/show/153394.S...,2008.0,/genres/young-adult|/genres/science-fiction|/g...,dir01/2767052-the-hunger-games.html,2958974,"The Hunger Games (The Hunger Games, #1)"
1,4.41,16648,0439358078,good_reads:book,https://www.goodreads.com/author/show/1077326....,2003.0,/genres/fantasy|/genres/young-adult|/genres/fi...,dir01/2.Harry_Potter_and_the_Order_of_the_Phoe...,1284478,Harry Potter and the Order of the Phoenix (Har...
2,3.56,85746,0316015849,good_reads:book,https://www.goodreads.com/author/show/941441.S...,2005.0,/genres/young-adult|/genres/fantasy|/genres/ro...,dir01/41865.Twilight.html,2579564,"Twilight (Twilight, #1)"
3,4.23,47906,0061120081,good_reads:book,https://www.goodreads.com/author/show/1825.Har...,1960.0,/genres/classics|/genres/fiction|/genres/histo...,dir01/2657.To_Kill_a_Mockingbird.html,2078123,To Kill a Mockingbird
4,4.23,34772,0679783261,good_reads:book,https://www.goodreads.com/author/show/1265.Jan...,1813.0,/genres/classics|/genres/fiction|/genres/roman...,dir01/1885.Pride_and_Prejudice.html,1388992,Pride and Prejudice


### Cleaning: Examing the dataframe - quick checks

We should examine the dataframe to get a overall sense of the content. 

<div class="exercise"><b>Exercise</b></div>
Lets check the types of the columns. What do you find?

In [10]:
df.dtypes

rating          float64
review_count     object
isbn             object
booktype         object
author_url       object
year            float64
genre_urls       object
dir              object
rating_count     object
name             object
dtype: object

*your answer here*

Notice that `review_count` and `rating_counts` are objects instead of ints, and the `year` is a float!

There are a couple more quick sanity checks to perform on the dataframe. 

In [11]:
print(df.shape)
df.columns

(6000, 10)


Index(['rating', 'review_count', 'isbn', 'booktype', 'author_url', 'year', 'genre_urls', 'dir', 'rating_count', 'name'], dtype='object')

### Cleaning: Examining the dataframe - a deeper look

Beyond performing checking some quick general properties of the data frame and looking at the first $n$ rows, we can dig a bit deeper into the values being stored. If you haven't already, check to see if there are any missing values in the data frame.

Let's see for a column which seemed OK to us.

In [12]:
#Get a sense of how many missing values there are in the dataframe.
print(np.sum([df.rating.isnull()]))
print(np.sum([df.review_count.isnull()]))
print(np.sum([df.isbn.isnull()]))
print(np.sum([df.booktype.isnull()]))
print(np.sum([df.author_url.isnull()]))
print(np.sum([df.year.isnull()]))
print(np.sum([df.genre_urls.isnull()]))
print(np.sum([df.dir.isnull()]))
print(np.sum([df.rating_count.isnull()]))
print(np.sum([df.name.isnull()]))

2
0
475
0
0
7
62
0
0
0


In [13]:
#Try to locate where the missing values occur
df[df.rating.isnull()]

,rating,review_count,isbn,booktype,author_url,year,genre_urls,dir,rating_count,name
3643,NaN,None,None,None,None,NaN,NaN,dir37/9658936-harry-potter.html,None,None
5282,NaN,None,None,None,None,NaN,NaN,dir53/113138.The_Winner.html,None,None


How does `pandas` or `numpy` handle missing values when we try to compute with data sets that include them?

We'll now check if any of the other suspicious columns have missing values.  Let's look at `year` and `review_count` first.

One thing you can do is to try and convert to the type you expect the column to be. If something goes wrong, it likely means your data are bad.

Lets test for missing data:

In [14]:
df[df.year.isnull()]

df.year.isnull()
df.shape

(6000, 10)

### Cleaning: Dealing with Missing Values
How should we interpret 'missing' or 'invalid' values in the data (hint: look at where these values occur)? One approach is to simply exclude them from the dataframe. Is this appropriate for all 'missing' or 'invalid' values? 

In [15]:
#Treat the missing or invalid values in your dataframe
####### 

df = df[df.year.notnull()]

Ok so we have done some cleaning. What do things look like now? Notice the float has not yet changed.

In [16]:
df.dtypes

rating          float64
review_count     object
isbn             object
booktype         object
author_url       object
year            float64
genre_urls       object
dir              object
rating_count     object
name             object
dtype: object

In [17]:
print(np.sum(df.year.isnull()))
df.shape # We removed seven rows

0


(5993, 10)

<div class="exercise"><b>Exercise</b></div>

Ok so lets fix those types. Convert them to ints. If the type conversion fails, we now know we have further problems.

In [18]:
# your code here
df.rating_count=df.rating_count.astype(int)
df.review_count=df.review_count.astype(int)
df.year=df.year.astype(int)

Once you do this, we seem to be good on these columns (no errors in conversion). Lets look:

In [19]:
df.dtypes

rating          float64
review_count      int64
isbn             object
booktype         object
author_url       object
year              int64
genre_urls       object
dir              object
rating_count      int64
name             object
dtype: object

Sweet!

Some of the other colums that should be strings have NaN.

In [20]:
df.loc[df.genre_urls.isnull(), 'genre_urls']=""
df.loc[df.isbn.isnull(), 'isbn']=""

##  Part 2: Parsing and Completing the Data Frame 

We will parse the `author` column from the author_url and `genres` column from the genre_urls. Keep the `genres` column as a string separated by '|'.

We will use panda's `map` to assign new columns to the dataframe.  

Examine an example `author_url` and reason about which sequence of string operations must be performed in order to isolate the author's name.

In [21]:
#Get the first author_url
test_string = df.author_url[0]
test_string

'https://www.goodreads.com/author/show/153394.Suzanne_Collins'

In [22]:
#Test out some string operations to isolate the author name

test_string.split('/')[-1].split('.')[1:][0]

'Suzanne_Collins'

<div class="exercise"><b>Exercise</b></div>

Lets wrap the above code into a function which we will then use

In [24]:
# Write a function that accepts an author url and returns the author's name based on your experimentation above
get_author = lambda url: url.split('/')[-1].split('.')[1:][0]

In [25]:
#Apply the get_author function to the 'author_url' column using '.map' 
#and add a new column 'author' to store the names
df['author'] = df.author_url.map(get_author)
df.author[0:5]

0    Suzanne_Collins
1        J_K_Rowling
2    Stephenie_Meyer
3         Harper_Lee
4        Jane_Austen
Name: author, dtype: object

<div class="exercise"><b>Exercise</b></div>

Now parse out the genres from `genre_url`.  

This is a little more complicated because there be more than one genre.


In [26]:
df.genre_urls.head()

0    /genres/young-adult|/genres/science-fiction|/g...
1    /genres/fantasy|/genres/young-adult|/genres/fi...
2    /genres/young-adult|/genres/fantasy|/genres/ro...
3    /genres/classics|/genres/fiction|/genres/histo...
4    /genres/classics|/genres/fiction|/genres/roman...
Name: genre_urls, dtype: object

In [28]:
# your code here
#Examine some examples of genre_urls

#Test out some string operations to isolate the genre name
test_genre_string=df.genre_urls[0]
genres=test_genre_string.strip().split('|')
for e in genres:
    print(e.split('/')[-1])
    "|".join(genres)

young-adult
science-fiction
dystopia
fantasy
science-fiction
romance
adventure
book-club
young-adult
teen
apocalyptic
post-apocalyptic
action


<div class="exercise"><b>Exercise</b></div>

Write a function that accepts a genre url and returns the genre name based on your experimentation above



In [30]:
split_and_join_genres = lambda url: "|".join([e.split('/')[-1] for e in url.strip().split('|')])

Test your function

In [31]:
split_and_join_genres("/genres/young-adult|/genres/science-fiction")

'young-adult|science-fiction'

<div class="exercise"><b>Exercise</b></div>

Use map again to create a new "genres" column

In [33]:

df['genres']=df.genre_urls.map(split_and_join_genres)
df.head()

,rating,review_count,isbn,booktype,author_url,year,genre_urls,dir,rating_count,name,author,genres
0,4.40,136455,0439023483,good_reads:book,https://www.goodreads.com/author/show/153394.S...,2008,/genres/young-adult|/genres/science-fiction|/g...,dir01/2767052-the-hunger-games.html,2958974,"The Hunger Games (The Hunger Games, #1)",Suzanne_Collins,young-adult|science-fiction|dystopia|fantasy|s...
1,4.41,16648,0439358078,good_reads:book,https://www.goodreads.com/author/show/1077326....,2003,/genres/fantasy|/genres/young-adult|/genres/fi...,dir01/2.Harry_Potter_and_the_Order_of_the_Phoe...,1284478,Harry Potter and the Order of the Phoenix (Har...,J_K_Rowling,fantasy|young-adult|fiction|fantasy|magic|chil...
2,3.56,85746,0316015849,good_reads:book,https://www.goodreads.com/author/show/941441.S...,2005,/genres/young-adult|/genres/fantasy|/genres/ro...,dir01/41865.Twilight.html,2579564,"Twilight (Twilight, #1)",Stephenie_Meyer,young-adult|fantasy|romance|paranormal|vampire...
3,4.23,47906,0061120081,good_reads:book,https://www.goodreads.com/author/show/1825.Har...,1960,/genres/classics|/genres/fiction|/genres/histo...,dir01/2657.To_Kill_a_Mockingbird.html,2078123,To Kill a Mockingbird,Harper_Lee,classics|fiction|historical-fiction|academic|s...
4,4.23,34772,0679783261,good_reads:book,https://www.goodreads.com/author/show/1265.Jan...,1813,/genres/classics|/genres/fiction|/genres/roman...,dir01/1885.Pride_and_Prejudice.html,1388992,Pride and Prejudice,Jane_Austen,classics|fiction|romance|historical-fiction|li...


Finally, let's pick an author at random so we can see the results of the transformations.  Scroll to see the `author` and `genre` columns that we added to the dataframe.

In [34]:
df[df.author == "Marguerite_Yourcenar"]

,rating,review_count,isbn,booktype,author_url,year,genre_urls,dir,rating_count,name,author,genres
1014,4.23,483,0374529264,good_reads:book,https://www.goodreads.com/author/show/7732.Mar...,1951,/genres/historical-fiction|/genres/fiction|/ge...,dir11/12172.Memoirs_of_Hadrian.html,6258,Memoirs of Hadrian,Marguerite_Yourcenar,historical-fiction|fiction|cultural|france|cla...
5620,4.11,74,2070367983,good_reads:book,https://www.goodreads.com/author/show/7732.Mar...,1968,/genres/fiction|/genres/historical-fiction|/ge...,dir57/953435.L_uvre_au_noir.html,1601,L'Åuvre au noir,Marguerite_Yourcenar,fiction|historical-fiction|cultural|france|eur...


Let us delete the `genre_urls` column.

In [35]:
del df['genre_urls']

And then save the dataframe out!

In [37]:
df.to_csv("../data/cleaned-goodreads.csv", index=False, header=True)

---

## Part 3: Grouping 

It appears that some books were written in negative years!  Print out the observations that correspond to negative years.  What do you notice about these books?  

In [38]:
# your code here
df[df.year < 0].name
#These are books written before the Common Era (BCE, equivalent to BC).

47                               The Odyssey
246                    The Iliad/The Odyssey
455                             The Republic
596                               The Aeneid
629                              Oedipus Rex
674                           The Art of War
746                        The Bhagavad Gita
777                                 Antigone
1233                       The Oedipus Cycle
1397                          Aesop's Fables
1398                   The Epic of Gilgamesh
1428                                   Medea
1815                            The Oresteia
1882         The Trial and Death of Socrates
2078    The History of the Peloponnesian War
2527                           The Histories
3133                          Complete Works
3274                  The Nicomachean Ethics
3757                              Lysistrata
4402                           The Symposium
4475                                 Apology
5367                          Five Dialogues
Name: name

We can determine the "best book" by year! For this we use Panda's `groupby()`. `Groupby()` allows grouping a dataframe by any (usually categorical) variable. Would it make sense to ever groupby integer variables? Floating point variables?

In [39]:
dfgb_author = df.groupby('author')
type(dfgb_author)

pandas.core.groupby.generic.DataFrameGroupBy

Perhaps we want the number of books each author wrote

In [40]:
dfgb_author.count()

,rating,review_count,isbn,booktype,author_url,year,dir,rating_count,name,genres
author,,,,,,,,,,
A_A_Milne,6,6,6,6,6,6,6,6,6,6
A_G_Howard,1,1,1,1,1,1,1,1,1,1
A_J_Cronin,1,1,1,1,1,1,1,1,1,1
A_J_Jacobs,1,1,1,1,1,1,1,1,1,1
A_J_Salt,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...
_,42,42,42,42,42,42,42,42,42,42
_gota_Krist_f,1,1,1,1,1,1,1,1,1,1
_mile_Zola,4,4,4,4,4,4,4,4,4,4


Lots of useless info there. One column should suffice

In [41]:
###### Before we start : what do we do about these titles where 'name' is unreadable? Try different encodings?
auth_name = 'A_id_al_Qarni'
df[df.author == auth_name].head()

,rating,review_count,isbn,booktype,author_url,year,dir,rating_count,name,author,genres
2213,4.19,1169,,good_reads:book,https://www.goodreads.com/author/show/1201952....,2003,dir23/2750180.html,15781,ÙØ§ ØªØ­Ø²Ù,A_id_al_Qarni,religion|religion|islam|self-help|non-fiction|...
5998,3.91,281,,good_reads:book,https://www.goodreads.com/author/show/1201952....,2006,dir60/2750008.html,3083,Ø£Ø³Ø¹Ø¯ Ø§Ù Ø±Ø£Ø© ÙÙ Ø§ÙØ¹Ø§ÙÙ,A_id_al_Qarni,religion|islam|religion|self-help|spirituality...


In [42]:
df[df.author == auth_name].iat[0,8].encode('UTF-16')

b'\xff\xfe\xd9\x00\x84\x00\xd8\x00\xa7\x00 \x00\xd8\x00\xaa\x00\xd8\x00\xad\x00\xd8\x00\xb2\x00\xd9\x00\x86\x00'

In [43]:
# let's examine the columns we have
df.columns

Index(['rating', 'review_count', 'isbn', 'booktype', 'author_url', 'year', 'dir', 'rating_count', 'name', 'author', 'genres'], dtype='object')

### Exercise:

- Group the dataframe by `author`. Include the following columns: `rating`, `name`, `author`. For the aggregation of the `name` column which includes the names of the books create a list with the strings containing the name of each book. Make sure that the way you aggregate the rest of the columns make sense! 

- Create a new column with number of books for each author and find the most prolific author!

Create the GroupBy table

In [44]:
authors = df.copy()
authors = authors[['rating','name','author']].groupby('author').agg({'rating' : np.mean,
                                                                    'name' : '|'.join})

In [46]:
authors = authors.reset_index()
authors.head()

,author,rating,name
0,A_A_Milne,4.365,Winnie-the-Pooh|The House at Pooh Corner|The H...
1,A_G_Howard,4.020,"Splintered (Splintered, #1)"
2,A_J_Cronin,4.220,The Keys of the Kingdom
3,A_J_Jacobs,3.750,The Year of Living Biblically
4,A_J_Salt,4.940,Nik Nassa & the Mark of Destiny


In [47]:
# split the column string and make a list of string book names
authors['name'] = authors.name.str.split('|')
authors.head()

,author,rating,name
0,A_A_Milne,4.365,"[Winnie-the-Pooh, The House at Pooh Corner, Th..."
1,A_G_Howard,4.020,"[Splintered (Splintered, #1)]"
2,A_J_Cronin,4.220,[The Keys of the Kingdom]
3,A_J_Jacobs,3.750,[The Year of Living Biblically]
4,A_J_Salt,4.940,[Nik Nassa & the Mark of Destiny]


In [48]:
# count the books - create new column
len(authors.name[0])

6

In [49]:
authors['num_books'] = authors['name'].str.len()
authors

,author,rating,name,num_books
0,A_A_Milne,4.365000,"[Winnie-the-Pooh, The House at Pooh Corner, Th...",6
1,A_G_Howard,4.020000,"[Splintered (Splintered, #1)]",1
2,A_J_Cronin,4.220000,[The Keys of the Kingdom],1
3,A_J_Jacobs,3.750000,[The Year of Living Biblically],1
4,A_J_Salt,4.940000,[Nik Nassa & the Mark of Destiny],1
...,...,...,...,...
2640,_,3.988095,"[Ø¹Ø²Ø§Ø²ÙÙ, Ø«ÙØ§Ø«ÙØ© ØºØ±ÙØ§Ø·Ø©, ØªØ±...",42
2641,_gota_Krist_f,4.340000,"[The Notebook, The Proof, The Third Lie]",1
2642,_mile_Zola,3.990000,"[Germinal (Les Rougon-Macquart, #13), L'Assomm...",4
2643,_ric_Emmanuel_Schmitt,4.160000,[Oscar et la dame rose],1


In [50]:
# sort for more prolific
authors.sort_values(by='num_books', ascending=False).iloc[0]

author                                            Stephen_King
rating                                                 3.91875
name         [The Stand, The Shining (The Shining #1), It, ...
num_books                                                   56
Name: 2349, dtype: object

#### Winner is Stephen King with 56 books! OMG!!!

Perhaps you want more detailed info...

In [51]:
dfgb_author[['rating', 'rating_count', 'review_count', 'year']].describe()

rating                                                        rating_count                                                                             review_count                                                                      year                                                                   
                       count      mean       std   min     25%    50%     75%   max        count          mean           std      min       25%      50%       75%       max        count         mean          std     min      25%     50%      75%     max count         mean         std     min      25%     50%      75%     max
author                                                                                                                                                                                                                                                                                                                                
A_A_Milne                6.0  4.365000  0.078677  4.25  4.3275  4.365  4.4025  4.48          6.0  47842.333333  57135.314010    544.0  15559.50  30547.0  50401.25  157833.0          6.0   543.000000   682.473150     1.0   219.50   309.5   499.25  1886.0   6.0  1944.166667   29.294482  1926.0  1926.25  1927.5  1952.75  1997.0
A_G_Howard               1.0  4.020000       NaN  4.02  4.0200  4.020  4.0200  4.02          1.0  17073.000000           NaN  17073.0  17073.00  17073.0  17073.00   17073.0          1.0  3194.000000          NaN  3194.0  3194.00  3194.0  3194.00  3194.0   1.0  2013.000000         NaN  2013.0  2013.00  2013.0  2013.00  2013.0
A_J_Cronin               1.0  4.220000       NaN  4.22  4.2200  4.220  4.2200  4.22          1.0   1015.000000           NaN   1015.0   1015.00   1015.0   1015.00    1015.0          1.0   109.000000          NaN   109.0   109.00   109.0   109.00   109.0   1.0  1941.000000         NaN  1941.0  1941.00  1941.0  1941.00  1941.0
A_J_Jacobs               1.0  3.750000       NaN  3.75  3.7500  3.750  3.7500  3.75          1.0  39489.000000           NaN  39489.0  39489.00  39489.0  39489.00   39489.0          1.0  4371.000000          NaN  4371.0  4371.00  4371.0  4371.00  4371.0   1.0  2007.000000         NaN  2007.0  2007.00  2007.0  2007.00  2007.0
A_J_Salt                 1.0  4.940000       NaN  4.94  4.9400  4.940  4.9400  4.94          1.0     16.000000           NaN     16.0     16.00     16.0     16.00      16.0          1.0     6.000000          NaN     6.0     6.00     6.0     6.00     6.0   1.0  2014.000000         NaN  2014.0  2014.00  2014.0  2014.00  2014.0
...                      ...       ...       ...   ...     ...    ...     ...   ...          ...           ...           ...      ...       ...      ...       ...       ...          ...          ...          ...     ...      ...     ...      ...     ...   ...          ...         ...     ...      ...     ...      ...     ...
_                       42.0  3.988095  0.301211  3.15  3.8000  4.075  4.2075  4.47         42.0  11574.642857   9368.218450     10.0   4724.00   8709.5  15572.75   39302.0         42.0  1437.214286  1216.421371     1.0   534.75  1020.5  1745.75  4785.0  42.0  1998.904762   18.721071  1941.0  1994.75  2007.0  2010.75  2014.0
_gota_Krist_f            1.0  4.340000       NaN  4.34  4.3400  4.340  4.3400  4.34          1.0   2628.000000           NaN   2628.0   2628.00   2628.0   2628.00    2628.0          1.0   243.000000          NaN   243.0   243.00   243.0   243.00   243.0   1.0  1986.000000         NaN  1986.0  1986.00  1986.0  1986.00  1986.0
_mile_Zola               4.0  3.990000  0.100995  3.84  3.9750  4.035  4.0500  4.05          4.0   8358.250000   4553.226942   3594.0   5181.75   8095.0  11271.50   13649.0          4.0   283.750000   171.069917   125.0   189.50   243.5   337.75   523.0   4.0  1652.500000  456.345264   968.0  1649.75  1878.5  1881.25  1885.0
_ric_Emmanuel_Schmitt    1.0  4.160000       NaN  4.16  4.1600  4.160  4.1600  4.16          1.0  

You can also access a `groupby` dictionary style.

In [53]:
ratingdict = {}
for author, subset in dfgb_author:
    ratingdict[author] = (subset['rating'].mean(), subset['rating'].std())
#ratingdict

<div class="exercise"><b>Exercise</b></div>

Lets get the best-rated book(s) for every year in our dataframe.

In [88]:
# Simple solution that takes 
df.sort_values(by = ['year','rating'], ascending = [True, True]). groupby('year').last().tail()['name']

year
2010                                  The Years Distilled
2011                                       Family Secrets
2012    Abstraction In Theory - Laws Of Physical Trans...
2013                                          Blade Heart
2014                 Untainted (Photographer Trilogy, #3)
Name: name, dtype: object